# Using AWS Autogluon to predict rent prices in Canada based on city, province, latitude, longitude, lease_term, type, price, beds, baths, sq_feet, furnishing, availability_date, smoking, and whether cats and dogs are allowed







In [ ]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce run

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("rentfaster.csv")

In [ ]:
df.head()

,rentfaster_id,city,province,address,latitude,longitude,lease_term,type,price,beds,baths,sq_feet,link,furnishing,availability_date,smoking,cats,dogs
0,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2495.0,2 Beds,2.5,1403,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,Immediate,Non-Smoking,True,True
1,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2695.0,3 Beds,2.5,1496,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,Immediate,Non-Smoking,True,True
2,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2295.0,2 Beds,2.5,1180,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,Immediate,Non-Smoking,True,True
3,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2095.0,2 Beds,2.5,1403,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,November 18,Non-Smoking,True,True
4,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2495.0,2 Beds,2.5,1403,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,Immediate,Non-Smoking,True,True


In [ ]:
df.columns

Index(['city', 'province', 'latitude', 'longitude', 'lease_term', 'type',
       'price', 'beds', 'baths', 'sq_feet', 'furnishing', 'availability_date',
       'smoking', 'cats', 'dogs'],
      dtype='object')

In [ ]:
# Remove unnecessary columns
df = df.drop(axis=1, columns = ["address", "link", "rentfaster_id"])

In [ ]:
df.head()

,city,province,latitude,longitude,lease_term,type,price,beds,baths,sq_feet,furnishing,availability_date,smoking,cats,dogs
0,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2495.0,2 Beds,2.5,1403,Unfurnished,Immediate,Non-Smoking,True,True
1,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2695.0,3 Beds,2.5,1496,Unfurnished,Immediate,Non-Smoking,True,True
2,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2295.0,2 Beds,2.5,1180,Unfurnished,Immediate,Non-Smoking,True,True
3,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2095.0,2 Beds,2.5,1403,Unfurnished,November 18,Non-Smoking,True,True
4,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2495.0,2 Beds,2.5,1403,Unfurnished,Immediate,Non-Smoking,True,True


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
# Regressing for the price
target = "price"

In [ ]:
train_data = TabularDataset(df)

In [ ]:
# Sample 70% randomly for the train data
subsample_size = int(0.7*len(df))

In [ ]:
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,city,province,latitude,longitude,lease_term,type,price,beds,baths,sq_feet,furnishing,availability_date,smoking,cats,dogs
12512,Calgary,Alberta,51.032613,-114.062190,Long Term,Condo Unit,2150.0,1 Bed,1,763,Unfurnished,July 01,Non-Smoking,False,False
24216,Montréal,Quebec,45.505681,-73.563915,Long Term,Apartment,3390.0,2 Beds,1,1058,Unfurnished,Immediate,Non-Smoking,True,True
13161,Calgary,Alberta,50.859881,-114.078010,Long Term,Basement,1300.0,1 Bed,1,700,Unfurnished,July 01,Non-Smoking,True,True
2415,Calgary,Alberta,51.134793,-113.949708,Long Term,Condo Unit,2150.0,2 Beds,2,980,Unfurnished,Immediate,Non-Smoking,False,False
7519,Edmonton,Alberta,53.544671,-113.577309,12 months,House,1700.0,2 Beds,2,800,Unfurnished,July 01,Non-Smoking,True,True


In [ ]:
# Training a fast model
predictor_price = TabularPredictor(label=target, path="agModels-predictprice").fit(train_data)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.12 GB / 12.67 GB (87.7%)
Disk Space Avail:   64.18 GB / 107.72 GB (59.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ..

[1000]	valid_set's rmse: 423.81
[2000]	valid_set's rmse: 415.834
[3000]	valid_set's rmse: 410.2
[4000]	valid_set's rmse: 407.48
[5000]	valid_set's rmse: 404.523
[6000]	valid_set's rmse: 402.728
[7000]	valid_set's rmse: 400.988
[8000]	valid_set's rmse: 399.787
[9000]	valid_set's rmse: 399.434
[10000]	valid_set's rmse: 398.293


	-398.293	 = Validation score   (-root_mean_squared_error)
	28.69s	 = Training   runtime
	6.34s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 378.305
[2000]	valid_set's rmse: 365.213
[3000]	valid_set's rmse: 360.246
[4000]	valid_set's rmse: 357.389
[5000]	valid_set's rmse: 355.923
[6000]	valid_set's rmse: 354.684
[7000]	valid_set's rmse: 353.801
[8000]	valid_set's rmse: 353.769
[9000]	valid_set's rmse: 353.433
[10000]	valid_set's rmse: 352.957


	-352.8283	 = Validation score   (-root_mean_squared_error)
	26.76s	 = Training   runtime
	3.41s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-375.0482	 = Validation score   (-root_mean_squared_error)
	25.15s	 = Training   runtime
	0.25s	 = Validation runtime
Fitting model: CatBoost ...
	-359.9289	 = Validation score   (-root_mean_squared_error)
	691.99s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-367.7976	 = Validation score   (-root_mean_squared_error)
	11.08s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-421.5468	 = Validation score   (-root_mean_squared_error)
	34.19s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: XGBoost ...
	-365.5962	 = Validation score   (-root_mean_squared_error)
	72.47s	 = Training   runtime
	0.96s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-523.5231	 = Validation score   (-root_mean_squared_error)
	31.13s	 = Training   runtime
	0.

[1000]	valid_set's rmse: 341.631
[2000]	valid_set's rmse: 336.496
[3000]	valid_set's rmse: 336.635


	-336.3645	 = Validation score   (-root_mean_squared_error)
	18.61s	 = Training   runtime
	1.76s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'LightGBMLarge': 0.429, 'RandomForestMSE': 0.143, 'CatBoost': 0.143, 'KNeighborsDist': 0.095, 'NeuralNetFastAI': 0.095, 'XGBoost': 0.095}
	-312.9316	 = Validation score   (-root_mean_squared_error)
	0.04s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 966.98s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 555.2 rows/s (1804 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("agModels-predictprice")


In [ ]:
# Creating test data for the rows that have not been sampled by the train data
test_data = TabularDataset(df.drop(train_data.index))
y_test = test_data[target]

In [ ]:
# predictor = TabularPredictor.load("agModels-predictprice")

# Predicting rent prices for the test data
y_pred = predictor_price.predict(test_data)
print("Predictions:  \n", y_pred)
perf = predictor_price.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Predictions:  
 10       2514.037109
11       2052.797852
13       2523.323730
19       2492.110840
24       1985.207031
            ...     
25748    1298.104858
25755    1064.368164
25762     930.289490
25763    1141.226807
25764     930.289490
Name: price, Length: 7732, dtype: float32


In [ ]:
# Looking at the performance
perf

{'root_mean_squared_error': -379.06860452004804,
 'mean_squared_error': -143693.0069327766,
 'mean_absolute_error': -169.37559000941798,
 'r2': 0.8486421928585806,
 'pearsonr': 0.9228787141770844,
 'median_absolute_error': -92.62921142578125}

In [ ]:
# retraining the model using best_quality and time limit of an hour, focusing on MAE
predictor_price = TabularPredictor(label=target, path="agModels-predictprice",
                                   eval_metric="mean_absolute_error").fit(train_data,
                                   presets="best_quality", time_limit=3600)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.03 GB / 12.67 GB (79.1%)
Disk Space Avail:   63.39 GB / 107.72 GB (58.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stack

(_ray_fit pid=13425) [1000]	valid_set's l1: 250.924
(_ray_fit pid=13425) [3000]	valid_set's l1: 235.046 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=13425) [4000]	valid_set's l1: 230.145 [repeated 2x across cluster]
(_ray_fit pid=13425) [6000]	valid_set's l1: 222.024 [repeated 4x across cluster]
(_ray_fit pid=13425) [8000]	valid_set's l1: 216.982 [repeated 4x across cluster]
(_ray_fit pid=13425) [10000]	valid_set's l1: 212.979 [repeated 4x across cluster]
(_ray_fit pid=13746) [1000]	valid_set's l1: 263.006 [repeated 2x across cluster]
(_ray_fit pid=13746) [3000]	valid_set's l1: 245.195 [repeated 4x across cluster]
(_ray_fit pid=13746) [5000]	valid_set's l1: 238.01 [repeated 4x across cluster]
(_ray_fit pid=13746) [7000]	valid_set's l1: 233.254 [repeated 4x across cluster]
(_ray_fit pid=13746) [90

(_dystack pid=13261) 	-226.9382	 = Validation score   (-mean_absolute_error)
(_dystack pid=13261) 	257.2s	 = Training   runtime
(_dystack pid=13261) 	107.51s	 = Validation runtime
(_dystack pid=13261) Fitting model: LightGBM_BAG_L1 ... Training model for up to 309.8s of the 606.2s of remaining time.
(_dystack pid=13261) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.07%)


(_ray_fit pid=14758) [1000]	valid_set's l1: 215.471 [repeated 2x across cluster]
(_ray_fit pid=14758) [2000]	valid_set's l1: 199.903 [repeated 2x across cluster]
(_ray_fit pid=14758) [4000]	valid_set's l1: 189.511 [repeated 4x across cluster]
(_ray_fit pid=14759) [6000]	valid_set's l1: 197.689 [repeated 4x across cluster]
(_ray_fit pid=14759) [8000]	valid_set's l1: 195.636 [repeated 4x across cluster]
(_ray_fit pid=14759) [10000]	valid_set's l1: 193.95 [repeated 4x across cluster]
(_ray_fit pid=15086) [1000]	valid_set's l1: 228.107 [repeated 2x across cluster]
(_ray_fit pid=15086) [3000]	valid_set's l1: 209.37 [repeated 4x across cluster]
(_ray_fit pid=15087) [4000]	valid_set's l1: 207.314 [repeated 3x across cluster]
(_ray_fit pid=15086) [6000]	valid_set's l1: 201.205 [repeated 3x across cluster]
(_ray_fit pid=15086) [8000]	valid_set's l1: 198.092 [repeated 4x across cluster]
(_ray_fit pid=15086) [10000]	valid_set's l1: 196.683 [repeated 4x across cluster]
(_ray_fit pid=15427) [1000]	

(_dystack pid=13261) 	-193.1719	 = Validation score   (-mean_absolute_error)
(_dystack pid=13261) 	264.15s	 = Training   runtime
(_dystack pid=13261) 	100.44s	 = Validation runtime
(_dystack pid=13261) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 26.32s of the 322.72s of remaining time.
(_dystack pid=13261) 	-186.8216	 = Validation score   (-mean_absolute_error)
(_dystack pid=13261) 	27.71s	 = Training   runtime
(_dystack pid=13261) 	0.91s	 = Validation runtime
(_dystack pid=13261) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 292.16s of remaining time.
(_dystack pid=13261) 	Ensemble Weights: {'RandomForestMSE_BAG_L1': 0.529, 'LightGBM_BAG_L1': 0.412, 'KNeighborsDist_BAG_L1': 0.059}
(_dystack pid=13261) 	-173.3011	 = Validation score   (-mean_absolute_error)
(_dystack pid=13261) 	0.08s	 = Training   runtime
(_dystack pid=13261) 	0.0s	 = Validation runtime
(_dystack pid=13261) Fitting 106 L2 models ...
(_dystack pid=13261) Fitting mo

(_ray_fit pid=16241) [1000]	valid_set's l1: 185.608 [repeated 3x across cluster]
(_ray_fit pid=16241) [3000]	valid_set's l1: 183.47 [repeated 4x across cluster]
(_ray_fit pid=16241) [4000]	valid_set's l1: 182.164 [repeated 2x across cluster]
(_ray_fit pid=16241) [6000]	valid_set's l1: 181.542 [repeated 4x across cluster]
(_ray_fit pid=16241) [7000]	valid_set's l1: 181.557 [repeated 2x across cluster]
(_ray_fit pid=16241) [9000]	valid_set's l1: 180.699 [repeated 4x across cluster]
(_ray_fit pid=16584) [1000]	valid_set's l1: 185.308 [repeated 4x across cluster]
(_ray_fit pid=16584) [2000]	valid_set's l1: 183.635 [repeated 2x across cluster]
(_ray_fit pid=16584) [4000]	valid_set's l1: 181.667 [repeated 2x across cluster]
(_ray_fit pid=16584) [5000]	valid_set's l1: 180.901
(_ray_fit pid=16584) [6000]	valid_set's l1: 180.506
(_ray_fit pid=16584) [7000]	valid_set's l1: 180.137
(_ray_fit pid=16584) [8000]	valid_set's l1: 179.849
(_ray_fit pid=16584) [9000]	valid_set's l1: 179.544
(_ray_fit pi

(_dystack pid=13261) 	-179.6073	 = Validation score   (-mean_absolute_error)
(_dystack pid=13261) 	156.92s	 = Training   runtime
(_dystack pid=13261) 	41.49s	 = Validation runtime
(_dystack pid=13261) Fitting model: LightGBM_BAG_L2 ... Training model for up to 123.09s of the 123.06s of remaining time.
(_dystack pid=13261) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.10%)
(_dystack pid=13261) 	-175.881	 = Validation score   (-mean_absolute_error)
(_dystack pid=13261) 	36.0s	 = Training   runtime
(_dystack pid=13261) 	0.69s	 = Validation runtime
(_dystack pid=13261) Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 83.55s of the 83.52s of remaining time.
(_dystack pid=13261) 	-161.1856	 = Validation score   (-mean_absolute_error)
(_dystack pid=13261) 	70.86s	 = Training   runtime
(_dystack pid=13261) 	1.02s	 = Validation runtime
(_dystack pid=13261) Fitting model: CatBoost_BAG_L2 ... Train

In [ ]:
test_data = TabularDataset(df.drop(train_data.index))
y_test = test_data[target]
# predictor = TabularPredictor.load("agModels-predictprice")

y_pred = predictor_price.predict(test_data)
print("Predictions:  \n", y_pred)
perf = predictor_price.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Predictions:  
 10       2625.232422
11       1909.255127
13       2749.968750
19       2288.382324
24       2052.271484
            ...     
25748    1304.355591
25755    1043.405640
25762     905.823364
25763    1110.603394
25764     905.823364
Name: price, Length: 7732, dtype: float32


In [ ]:
# Take a look at the performance of the model
perf

{'mean_absolute_error': -153.4067275796594,
 'root_mean_squared_error': -380.5164302788771,
 'mean_squared_error': -144792.75371217958,
 'r2': 0.8474837839387982,
 'pearsonr': 0.9212355570349564,
 'median_absolute_error': -72.9281005859375}

In [ ]:
pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

,y_test,y_pred
10,1930.0,2625.232422
11,1700.0,1909.255127
13,3150.0,2749.968750
19,2300.0,2288.382324
24,1910.0,2052.271484
...,...,...
25748,1305.0,1304.355591
25755,1085.0,1043.405640
25762,945.0,905.823364
25763,1025.0,1110.603394


In [ ]:
from google.colab import files
files.download('/content/agModels-predictprice/models/WeightedEnsemble_L3')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>